# How to work with your azimuthally binned data

You can use this notebook as inspiration to help you process your azimuthally binned data and quickly prepare plots

### Content:
[**Import data**](#import)  
[**Plot average diffraction pattern**](#plot_pxrd)  
[**Plot azimuthal cake**](#plot_cake)  
[**Plot azimuthal cake with variable azimuthal steps**](#plot_cake_2)  

In [ ]:
%matplotlib widget
import DanMAX as DM
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
print('Current visit directory:')
print(f"'{os.getcwd().split('/scripts')[0]}'")

#### Import data <a id='import'></a>

In [ ]:
# Insert path for the .h5 file - TIP: Use tap for auto-complete
#fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/scan-####.h5'
fname = DM.getLatestScan()

# get the azimuthally integrated filename from master file name
azint_fname = fname.replace('raw', 'process/azint').split('.')[0]+'_bin_integrated.h5'


with h5py.File(azint_fname, 'r') as af:
    azint_data = af['I'][:]
    azint_angle = af['phi'][:]
    # Determine if the file is integrated in tth or q
    try:
        r_unit = af['2th'][:]
        tth = True
    except KeyError:
        r_unit = af['q'][:]
        tth = False

azint_data[azint_data<=0]= np.nan

# read common meta data from the master file
meta = DM.getMetaData(fname)
#meta_dic = DM.getMetaDic(fname) # extended meta data dictionary
t = meta['time'] # relative time stamp in seconds
T = meta['temp'] # temperature in Kelvin (if available, otherwise None)
I0 = meta['I0']  # relative incident beam intensity "I zero"
E = meta['energy'] # X-ray energy in keV


#### Plot average diffraction pattern <a id='plot_pxrd'></a>

In [ ]:
frame = 0
plt.figure()
plt.suptitle(f'Average diffraction pattern - Frame {frame:5d}')
plt.plot(r_unit, np.nanmean(azint_data[frame, :,:],axis=0))
plt.ylabel('I [a.u.]')
if tth:
    plt.xlabel(r'2$\theta$ [$\deg$]')
else:
    plt.xlabel(r'Q [$\AA^{-1}$]')

In [ ]:
def getVmax(im, limit=0.99,bins=1000):
    """
    Return vmax (float) corresponding to the largest value below a defined limit for the cumulative histogram
    NB: Should be run prior to initializing the figure, as it closes the current pyplot figure.
    """
    plt.close()
    h = plt.hist(im[im>0],bins=bins,density=True,cumulative=True)
    vargmax = np.argmax(h[0][h[0]<limit])
    plt.close()
    return h[1][vargmax]
vmax = getVmax(azint_data)

#### Plot azimuthal cake <a id='plot_cake'></a>  
Simpel framewise heatmap of the azimuthal cake. Assumes equidistant azimuthal steps.

In [ ]:
frame = 0
# Generate and show the waterfall plot
plt.figure()
plt.suptitle(f'Azimuthal cake - Frame {frame:5d}')
plt.imshow(azint_data[frame,:,:],
           extent = [r_unit[0],r_unit[-1],azint_angle[-1],azint_angle[0]],
           interpolation = 'Nearest',
           aspect = 'auto',
           vmin = 0,
           vmax = vmax)
plt.ylabel(r'Azimuthal angle [$\deg$]')
if tth:
    plt.xlabel(r'2$\theta$ [$\deg$]')
else:
    plt.xlabel(r'Q [$\AA^{-1}$]')

#### Plot azimuthal cake with variable azimuthal steps <a id='plot_cake_2'></a>  
Simpel framewise heatmap of the azimuthal cake. Allows non-equidistant azimuthal steps.

In [ ]:
frame = 0
# Generate and show the waterfall plot
plt.figure()
plt.suptitle(f'Azimuthal cake - Frame {frame:5d}')
plt.pcolormesh(r_unit, azint_angle, azint_data[frame,:,:],
               shading = 'Nearest',
               vmin = 0,
               vmax = vmax,
               cmap = 'cividis')
plt.ylabel(r'Azimuthal angle [$\deg$]')
if tth:
    plt.xlabel(r'2$\theta$ [$\deg$]')
else:
    plt.xlabel(r'Q [$\AA^{-1}$]')